In [3]:
!pip install -q pyomo

     |████████████████████████████████| 9.7 MB 25.7 MB/s 
     |████████████████████████████████| 49 kB 6.9 MB/s 


In [4]:
from pyomo.environ import * 

In [5]:
import numpy as np

In [6]:
coef = np.loadtxt('lab4_ex1_coef.txt', delimiter=',')

In [7]:
print(coef.shape)
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])


(6, 7)
Num rows: 6
Num cols: 7


In [8]:
print(coef)

[[  0.65   0.75  -5.85  -8.35 -12.38  30.     0.  ]
 [ -0.5    0.    -3.8   -0.9    0.9    0.    24.8 ]
 [  1.     0.     0.    -1.     0.    -1.     0.8 ]
 [  0.    -2.9   -1.9    0.     0.    -0.9  -71.4 ]
 [  0.     0.     0.     4.7    8.02   6.15  95.  ]
 [ -0.7   -1.4    7.9   -2.9    0.9    0.   -47.  ]]


In [9]:
model = ConcreteModel()

In [10]:
N = coef.shape[1]-1

In [11]:
M = coef.shape[0]-1

In [12]:
obj_coef = coef[0,:-1]
print(obj_coef.shape)
print(obj_coef)


(6,)
[  0.65   0.75  -5.85  -8.35 -12.38  30.  ]


In [13]:
constr_coef = coef[1:,:-1]
print(constr_coef.shape)
print(constr_coef)

(5, 6)
[[-0.5   0.   -3.8  -0.9   0.9   0.  ]
 [ 1.    0.    0.   -1.    0.   -1.  ]
 [ 0.   -2.9  -1.9   0.    0.   -0.9 ]
 [ 0.    0.    0.    4.7   8.02  6.15]
 [-0.7  -1.4   7.9  -2.9   0.9   0.  ]]


In [14]:
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(5,)
[ 24.8   0.8 -71.4  95.  -47. ]


In [15]:
lower_bound = 0
upper_bound = np.inf

In [16]:
col_indices = np.arange(N)
print(col_indices)

[0 1 2 3 4 5]


In [17]:
model.x = Var(col_indices)

In [18]:
row_indices = np.arange(M)
print(row_indices)

[0 1 2 3 4]


In [19]:
model.constraints = ConstraintList()


In [20]:
for i in row_indices:
  model.constraints.add(summation(constr_coef[i],model.x) <= constr_rhs[i])

In [21]:
for j in col_indices:
  model.x[j].setlb(lower_bound)
  model.x[j].setub(upper_bound)

In [22]:
model.objective = Objective(expr = summation(obj_coef,model.x), sense=maximize)

In [23]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals
          5 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.65*x[0] + 0.75*x[1] - 5.85*x[2

In [24]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [25]:
opt_cbc = SolverFactory('cbc')

In [26]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: unbounded
  - message from solver: <undefined>


Solver status: warning
Solver termination condition: unbounded


In [27]:
model.objective.set_sense(minimize)

In [28]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals
          5 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.65*x[0] + 0.75*x[1] - 5.85*x[2

In [29]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: unbounded
  - message from solver: <undefined>


Solver status: warning
Solver termination condition: unbounded


Answer1(a):

 $ \\ $ Since in above both cases when the sense of  objective function was maximum and minimum respectively then the solver termination condtion is unbounded.Reason behind getting such kind of solver termination condition is variables are not bounded here.Bacause of this reason value of objective function will be unbounded here.
 Since constraint 3 has negative coefficients and it is unbounded below.And variables are bounded below and unbounded above. To make constraint bounded above,value of variable must be low in value so that variables must be bounded above.
  So now to obtain an optimal solution we bound the variables.Let all the variables are bounded below by 0 and and bounded above by 25.

In [45]:
model.objective.set_sense(maximize)

In [46]:
lower_bound_1 = 0
upper_bound_1 = 25

In [47]:
for j in col_indices:
  model.x[j].setlb(lower_bound_1)
  model.x[j].setub(upper_bound_1)

In [48]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value     : Upper : Fixed : Stale : Domain
          0 :     0 :       0.0 :    25 : False : False :  Reals
          1 :     0 :      25.0 :    25 : False : False :  Reals
          2 :     0 : 5.9008888 :    25 : False : False :  Reals
          3 :     0 : 20.212766 :    25 : False : False :  Reals
          4 :     0 :       0.0 :    25 : False : False :  Reals
          5 :     0 :       0.0 :    25 : False : False :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.65

In [49]:
!apt-get install -y -qq coinor-cbc

In [50]:
opt_cbc = SolverFactory('cbc')

In [51]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [52]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()


Objective =  486.3585647945

Decision Variables
x[ 0 ] =  16.295385
x[ 1 ] =  25.0
x[ 2 ] =  0.0
x[ 3 ] =  0.20456233
x[ 4 ] =  0.0
x[ 5 ] =  15.290822

Constraints
constraints : Size=5
    Key : Lower : Body                : Upper
      1 :  None :        -8.331798597 :  24.8
      2 :  None :  0.8000006699999975 :   0.8
      3 :  None :         -86.2617398 : -71.4
      4 :  None :   94.99999825100001 :  95.0
      5 :  None : -47.000000256999996 : -47.0


Answer:
Hence when we bounded variables by 0 and 25 then we get an optimal solution.And to obtain this we also changed the sense minimize.

Objective =  -320.7335045

Decision Variables
$ x_1 =  0.0,
x_2 =  500.0,
x_3 =  90.078104,
x_4 =  20.212766,
x_5 =  0.0,
x_6 =  0.0 $

Constraint 1 is not active since LHS = -360.4882846$ \neq RHS \\  $

Constraint 2 is not active since LHS =  -20.212766 $\neq RHS \\  $

Constraint 3 is not active since LHS =  -1621.1483976$ \neq RHS \\  $

Constraint 4 is active since LHS =  95.0000002$ \approx RHS \\  $

Constraint 5 is active since LHS = -46.99999979999995 $ \approx RHS  $



